# Chapter 9 딥러닝의 미래 GAN

GAN(Generative Adversarial Network)는 서로 대립하는 두 신경망을 경쟁시켜가며 결과물을 생성하는 방법을 학습하는 모델입니다.

> 위조지폐범(생성자)와 경찰(구분자)가 있다고 가정합시다. 위조지폐범은 경찰을 속이려고 노력하고, 경찰은 위조한 지폐를 감별하려고 최대한 노력합니다. 이를 통해 서로의 능력이 발전하고, 위조지폐범은 진짜와 거의 구별되지 않는 위조지폐를 만들 수 있게 됩니다.

신경망 모델로 나타내면 아래와 같습니다

![GAN_1](GAN_1.png)

1. 먼저 실제 이미지를 주고 구분자(Discriminator)에게 이 이미지가 진짜임을 판단하게 합니다.
2. 생성자(Generator)를 통해 임의의 이미지를 만들고 이것을 다시 같은 구분자를 통해 진짜 이미지인지를 판단하게 합니다.
3. 생성자는 구분자를 속여 진짜처럼 보이게 하고, 구분자는 생성자가 만든 이미지를 최대한 가짜라고 구분하도록 훈련한다.

이런 경쟁을 통해 생성자는 실제 이미지와 상당히 비슷한 이미지를 생성하게 된다.

1. 사진을 고흐 풍 그림으로 다시 그리기
2. 선으로만 그려진 만화를 채색
3. 모자이크 없애기
4. 자연어 문장 생성 등등

![GAN_2](GAN_2.png)

이번 장에서는 GAN 모델을 이용해 MNIST 손글씨를 무작위로 생성하고, 이를 학습하여 원하는 숫자의 이미지를 생성하는 모델을 만들어보겠습니다.

## 9.1 GAN 기본 모델 구현하기



In [18]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [19]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 255
n_input = 28 * 28
n_noise = 128 # 생성자의 입력값으로 사용할 노이즈의 크기

In [20]:
# GAN도 비지도 학습이므로 오토인코더처럼 Y(Labeling) 데이터를 사용하지 않습니다.

# 이미지를 넣을 X와 이미지에 들어가는 노이즈를 넣을 Z를 추가한다.
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [21]:
# 생성자 신경망에 사용할 변수 설정
# 노이즈로 이미지를 생성
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [22]:
# 구분자 신경망에 사용할 변수 설정
# 이미지를 넣고 가짜인지 진짜인지 판별
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

구분자는 실제 이미지와 생성한 이미지를 받아 가짜인지를 구별한다

In [23]:
# 생성자 신경망
# 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 
# 실제 이미지와 같은 크기의 가짜 결괏값을 반환한다.
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [24]:
# 구분자 신경망
# 0~1 사이의 진짜/가짜 여부를 판단한 결과를 반환
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [25]:
# 무작위한 노이즈를 만들어주는 유틸리티 함수 정의
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [26]:
# 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만들고
G = generator(Z)
# G가 만든 가짜 이미지와 진짜 이미지 X를 각각 구분자에 넣어
# 입력한 이미지가 진짜인지를 판별하도록 합니다.
D_gene = discriminator(G)
D_real = discriminator(X)

이제 손실값을 구합니다.

1. 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값(경찰 학습용)
2. 진짜라고 판단하도록 하는 손실값(위조지폐범 학습용)

경찰(discriminator)를 학습시키려면 

1. 진짜 이미지 판별값 D_real은 1에 가까워야 하고
2. 가짜 이미지 판별값 D_gene은 0에 가까워야 합니다

In [27]:
# D-real은 1에 가까워질 수록 좋고, D_gene은 0에 가까울 수록 좋음
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

위조지폐범 학습은 가짜 이미지 판별값 D-gene을 1에 가깝게 만들면 됩니다.

- 즉 가짜 이미지를 진짜라고 판별해야 합니다.

In [28]:
loss_G = tf.reduce_mean(tf.log(D_gene))

GAN의 학습은 loss_D와 loss_G 모두를 최대화하는 것입니다.

- 다만 이 둘은 서로 연관되어 있어서 항상 같이 증가하지는 않습니다.
- 경찰이 잘하면 위조지폐범이 실패하고, 반대의 경우에도 마찬가지이기 때문

![GAN_3](GAN_3.png)

In [29]:
# 학습할 때에 구분자와 생성자 각각의 변수만 학습되도록 변수로 넘겨줌
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [30]:
# loss값을 최대화 하기 위해 음수를 붙여서 학습합니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_G, var_list=G_var_list)

In [31]:
# 지금까지는 손실값을 1개만 학습시켰지만 이번에는 2개를 학습시킨다는 점이 다릅니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_var_D, loss_var_G = 0, 0

In [32]:
# 미니배치로 학습을 반복합니다.
# 구분자는 X 값을,
# 생성자는 노이즈인 Z 값을 받습니다.

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Z: noise})
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)
print('최적화 완료!')
    

Epoch: 0000 D loss: -0.5403 G loss: -2.055
Epoch: 0001 D loss: -0.3983 G loss: -2.407
Epoch: 0002 D loss: -0.1097 G loss: -3.156
Epoch: 0003 D loss: -0.4275 G loss: -1.488
Epoch: 0004 D loss: -0.3924 G loss: -1.939
Epoch: 0005 D loss: -0.2872 G loss: -2.873
Epoch: 0006 D loss: -0.2407 G loss: -2.346
Epoch: 0007 D loss: -0.2423 G loss: -2.797
Epoch: 0008 D loss: -0.2207 G loss: -2.756
Epoch: 0009 D loss: -0.4506 G loss: -2.201
Epoch: 0010 D loss: -0.3987 G loss: -2.202
Epoch: 0011 D loss: -0.3482 G loss: -2.234
Epoch: 0012 D loss: -0.3926 G loss: -2.337
Epoch: 0013 D loss: -0.4399 G loss: -2.416
Epoch: 0014 D loss: -0.3892 G loss: -2.414
Epoch: 0015 D loss: -0.4375 G loss: -2.216
Epoch: 0016 D loss: -0.5471 G loss: -2.077
Epoch: 0017 D loss: -0.4305 G loss: -2.202
Epoch: 0018 D loss: -0.4553 G loss: -2.365
Epoch: 0019 D loss: -0.2835 G loss: -2.798
Epoch: 0020 D loss: -0.4252 G loss: -2.297
Epoch: 0021 D loss: -0.4786 G loss: -2.463
Epoch: 0022 D loss: -0.4576 G loss: -2.516
Epoch: 0023

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 255
n_input = 28 * 28
n_noise = 128 # 생성자의 입력값으로 사용할 노이즈의 크기

# 이미지를 넣을 X와 이미지에 들어가는 노이즈를 넣을 Z를 추가한다.
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

train_D = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_G, var_list=G_var_list)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_var_D, loss_var_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Z: noise})
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)
print('최적화 완료!')
    

## 9.2 원하는 숫자 생성하기

이번에는 숫자를 무작위로 생성하지 않고 원하는 숫자를 지정해 생성하는 모델을 만들어보겠습니다.

- 여기서는 간단하게 노이즈에 레이블 데이터를 힌트로 넣어주겠습니다.

In [35]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

total_epoch = 100
batch_size = 100
n_hidden = 255
n_input = 28 * 28
n_noise = 128 
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈와 실제 이미지에 해당하는 숫자를 힌트로 넣어주는 용도로 사용
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


생성자 신경망을 구성하는데 이번에는 변수를 선언하지 않고 `tf.layers`를 사용합니다.

In [36]:
def generator(noise, labels):
    # 학습하는 변수를 지정할 수 있음
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden,
                                activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,
                                activation=tf.nn.sigmoid)
    return output

def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 진짜 이미지를 판별할 때와 가짜 이미지를 판별할 때 똑같은 변수를 사용해야 함
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden,
                                activation=tf.nn.relu)
        # 손실값 계산에 cross_entropy를 사용하기 위해 여기서는 사용하지 않음
        output = tf.layers.dense(hidden, 1,
                                activation=None)

    return output

def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [37]:
G = generator(Z, Y)
D_real = discriminator(X, Y)
# 가짜 이미지 구분자를 만들 때는 진짜 이미지 구분자에서 사용한 변수들을 재사용하도록
# reuse 옵션을 True로 설정
D_gene = discriminator(G, Y, True)

In [38]:
# 구분자(discriminator)의 손실 함수를 만들 차례입니다.
# D_real은 1에 가까워야 합니다. 
# ones_like를 통해 1로 채워진 값과 비교합니다
loss_D_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_real, labels=tf.ones_like(D_real)))
# D_gene은 0에 가까워야 합니다. 
# ones_like를 통해 0으로 채워진 값과 비교합니다
loss_D_gene = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_gene, labels=tf.zeros_like(D_gene)))

# 이 값을 최소화하면 경찰을 학습시킬 수 있습니다.
loss_D = loss_D_real + loss_D_gene

In [39]:
# 생성자(generator)의 손실 함수를 만들 차례입니다.
# 생성자의 D_gene은 1에 가까워야 합니다.
loss_G = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_gene, labels=tf.ones_like(D_gene)))

In [40]:
# variable_scope에서 사용된 변수들을 가져와 최적화 함수에 전달
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                            var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                            var_list=vars_G)

In [42]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Y: batch_ys, Z: noise})
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size],
                                         Z: noise})
        
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)
print('최적화 완료!')
    

Epoch: 0000 D loss: 0.005344 G loss: 7.659
Epoch: 0001 D loss: 0.008757 G loss: 9.786
Epoch: 0002 D loss: 0.01028 G loss: 8.461
Epoch: 0003 D loss: 0.0194 G loss: 6.601
Epoch: 0004 D loss: 0.0136 G loss: 8.135
Epoch: 0005 D loss: 0.06313 G loss: 6.373
Epoch: 0006 D loss: 0.05157 G loss: 6.581
Epoch: 0007 D loss: 0.07335 G loss: 7.153
Epoch: 0008 D loss: 0.1555 G loss: 6.3
Epoch: 0009 D loss: 0.1197 G loss: 6.685
Epoch: 0010 D loss: 0.406 G loss: 4.718
Epoch: 0011 D loss: 0.3326 G loss: 5.213
Epoch: 0012 D loss: 0.3472 G loss: 5.015
Epoch: 0013 D loss: 0.3094 G loss: 4.307
Epoch: 0014 D loss: 0.5169 G loss: 3.894
Epoch: 0015 D loss: 0.5524 G loss: 3.958
Epoch: 0016 D loss: 0.5442 G loss: 3.523
Epoch: 0017 D loss: 0.3521 G loss: 3.856
Epoch: 0018 D loss: 0.4923 G loss: 3.285
Epoch: 0019 D loss: 0.5635 G loss: 3.403
Epoch: 0020 D loss: 0.3957 G loss: 3.183
Epoch: 0021 D loss: 0.5347 G loss: 3.021
Epoch: 0022 D loss: 0.7371 G loss: 3.186
Epoch: 0023 D loss: 0.595 G loss: 2.658
Epoch: 0024 